In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import re
pd.set_option('display.max_columns', None)  # Alle Spalten anzeigen

## Merge Daten und und METAR

In [2]:
from pathlib import Path
import pandas as pd

# ==========================================
# 1) Einstellungen & Pfade
# ==========================================
P_METAR   = Path("metar_official_HALBHOUR_MATCHED.csv")
P_FLIGHTS = Path("flights_with_meteosuisse_HALBHOUR_merged.csv")
P_OUTPUT  = Path("flights_with_weather_HALBHOUR_merged.csv")

GRID_START = pd.Timestamp("2023-01-01 05:50", tz="UTC")
STEP_MIN   = 30

# ==========================================
# 2) Hilfsfunktionen
# ==========================================
def to_utc_naive(s: pd.Series) -> pd.Series:
    x = pd.to_datetime(s, errors="coerce", utc=True)
    return x.dt.tz_convert("UTC").dt.tz_localize(None)

def snap_to_grid(ts: pd.Series, grid_start: pd.Timestamp, step_min: int = 30) -> pd.Series:
    if isinstance(grid_start, pd.Timestamp) and grid_start.tz is not None:
        ts_aware = pd.to_datetime(ts, errors="coerce", utc=True)
        ts_naive = ts_aware.dt.tz_convert("UTC").dt.tz_localize(None)
        gs_naive = grid_start.tz_convert("UTC").tz_localize(None)
    else:
        ts_naive = pd.to_datetime(ts, errors="coerce")
        gs_naive = pd.Timestamp(grid_start)
    delta = ts_naive - gs_naive
    step  = pd.Timedelta(minutes=step_min)
    floored = (delta // step) * step
    return gs_naive + floored

# ==========================================
# 3) Daten laden
# ==========================================
metar = pd.read_csv(P_METAR, encoding="utf-8")
flights = pd.read_csv(P_FLIGHTS, encoding="utf-8")

# ==========================================
# 4) Zeitspalten harmonisieren
# ==========================================
if "time" not in metar.columns:
    raise ValueError("Im METAR-File wird eine Spalte 'time' erwartet.")
metar["time"] = to_utc_naive(metar["time"])
metar = metar.rename(columns={"time": "grid_time"}).copy()

if "grid_time" in flights.columns:
    flights["grid_time"] = to_utc_naive(flights["grid_time"])
else:
    if "interval_start" not in flights.columns:
        raise ValueError("Im Flights-File wird entweder 'grid_time' oder 'interval_start' benötigt.")
    flights["interval_start"] = to_utc_naive(flights["interval_start"])
    flights["grid_time"] = snap_to_grid(flights["interval_start"], GRID_START, STEP_MIN)

# doppelte Zeiten entfernen
flights = flights.sort_values("grid_time").drop_duplicates(subset=["grid_time"], keep="last")
metar   = metar.sort_values("grid_time").drop_duplicates(subset=["grid_time"], keep="last")

# ==========================================
# 5) Merge: Flights als Referenz (alle Zeilen behalten)
#    METAR-Daten werden rechts angefügt
# ==========================================
merged = flights.merge(
    metar,
    on="grid_time",
    how="left",
    suffixes=("", "_metar")
)

# ==========================================
# 6) Optional: METAR-Spalten ans Ende verschieben
# ==========================================
# (sorgt dafür, dass grid_time + alle Flights vorne bleiben)
metar_cols = [c for c in merged.columns if c.endswith("_metar") or c in metar.columns and c != "grid_time"]
flight_cols = [c for c in merged.columns if c not in metar_cols]
merged = merged[flight_cols + metar_cols]

# ==========================================
# 7) Diagnose
# ==========================================
print("\n✅ Merge abgeschlossen")
print(f"Flights-Zeilen: {len(flights):,}")
print(f"METAR-Zeilen:   {len(metar):,}")
print(f"Output-Zeilen:  {len(merged):,}")
missing_rows = merged.isna().all(axis=1).sum()
print(f"Vollständig leere Zeilen: {missing_rows:,}")

# ==========================================
# 8) Speichern
# ==========================================
merged.to_csv(P_OUTPUT, index=False, encoding="utf-8")
print(f"\n💾 Datei gespeichert als: {P_OUTPUT}")
print("\nVorschau:")
print(merged.head(8))



✅ Merge abgeschlossen
Flights-Zeilen: 47,316
METAR-Zeilen:   47,316
Output-Zeilen:  47,316
Vollständig leere Zeilen: 0

💾 Datei gespeichert als: flights_with_weather_HALBHOUR_merged.csv

Vorschau:
        interval_start         interval_end  n_arrivals  n_departures  \
0  2023-01-01 05:50:00  2023-01-01 06:20:00           5             0   
1  2023-01-01 06:20:00  2023-01-01 06:50:00           1             3   
2  2023-01-01 06:50:00  2023-01-01 07:20:00           4             9   
3  2023-01-01 07:20:00  2023-01-01 07:50:00           1            18   
4  2023-01-01 07:50:00  2023-01-01 08:20:00           5             8   
5  2023-01-01 08:20:00  2023-01-01 08:50:00           8             1   
6  2023-01-01 08:50:00  2023-01-01 09:20:00           1             1   
7  2023-01-01 09:20:00  2023-01-01 09:50:00           4             3   

   n_total           grid_time station  \
0        5 2023-01-01 05:50:00     KLO   
1        4 2023-01-01 06:20:00     KLO   
2       13 2023-01

In [3]:
df = pd.read_csv("flights_with_weather_HALBHOUR_merged.csv")
df

,interval_start,interval_end,n_arrivals,n_departures,n_total,grid_time,station,air_temperature_2_m_above_ground_current_value_[degc],air_temperature_at_5_cm_above_grass_current_value_[degc],air_temperature_at_surface_current_value_[degc],chill_temperature_current_value_[degc],relative_air_humidity_2_m_above_ground_current_value_[percent],dew_point_2_m_above_ground_current_value_[degc],vapour_pressure_2_m_above_ground_current_value_[hpa],atmospheric_pressure_at_barometric_altitude_(qfe)_current_value_[hpa],pressure_reduced_to_sea_level_according_to_standard_atmosphere_(qnh)_current_value_[hpa],pressure_reduced_to_sea_level_(qff)_current_value_[hpa],geopotential_height_of_the_850_hpa_level_current_value_[gpm],geopotential_height_of_the_700_hpa_level_current_value_[gpm],gust_peak_(one_second)_maximum_in_m_per_s_[m_per_s],wind_speed_vectoriel_ten_minutes_mean_in_m_per_s_[m_per_s],wind_speed_scalar_ten_minutes_mean_in_m_per_s_[m_per_s],wind_direction_ten_minutes_mean_[deg],foehn_index_[code],wind_speed_ten_minutes_mean_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_m_per_s_[m_per_s],gust_peak_(one_second)_maximum_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_km_per_h_[km_per_h],precipitation_ten_minutes_total_[mm],snow_depth_(automatic_measurement)_current_value_[cm],global_radiation_ten_minutes_mean_[w_per_m²],diffuse_radiation_ten_minutes_mean_[w_per_m²],longwave_incoming_radiation_ten_minutes_mean_[w_per_m²],longwave_outgoing_radiation_ten_minute_mean_[w_per_m²],shortwave_reflected_radiation_ten_minute_mean_[w_per_m²],sunshine_duration_ten_minutes_total_[min],code,type,correction,mod,station_id,cycle,wind_dir,wind_speed,wind_gust,wind_dir_from,wind_dir_to,vis,vis_dir,max_vis,max_vis_dir,temp,dewpt,press,runway,weather,recent,sky,windshear,wind_speed_peak,wind_dir_peak,peak_wind_time,wind_shift_time,max_temp_6hr,min_temp_6hr,max_temp_24hr,min_temp_24hr,press_sea_level,precip_1hr,precip_3hr,precip_6hr,precip_24hr,snowdepth,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,airport
0,2023-01-01 05:50:00,2023-01-01 06:20:00,5,0,5,2023-01-01 05:50:00,KLO,13.0,9.9,9.1,13.0,66.3,6.9,9.9,973.0,1023.8,1023.5,NaN,NaN,7.3,4.0,4.1,224.0,NaN,14.8,6.9,26.3,24.8,0.0,0.0,2,1.0,302,NaN,NaN,0,LSZH 010550Z 22007KT CAVOK 13/07 Q1023 NOSIG,METAR,NaN,AUTO,LSZH,6.0,220 degrees,7 knots,NaN,NaN,NaN,10000 meters,NaN,NaN,NaN,13.0 C,7.0 C,1023.0 mb,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSZH
1,2023-01-01 06:20:00,2023-01-01 06:50:00,1,3,4,2023-01-01 06:20:00,KLO,12.5,9.7,9.0,12.5,68.4,6.8,9.9,973.2,1024.0,1023.8,NaN,NaN,6.1,4.1,4.1,233.0,NaN,14.8,5.7,22.0,20.5,0.0,0.0,2,1.0,300,NaN,NaN,0,LSZH 010620Z 24007KT CAVOK 13/07 Q1024 NOSIG,METAR,NaN,AUTO,LSZH,6.0,240 degrees,7 knots,NaN,NaN,NaN,10000 meters,NaN,NaN,NaN,13.0 C,7.0 C,1024.0 mb,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSZH
2,2023-01-01 06:50:00,2023-01-01 07:20:00,4,9,13,2023-01-01 06:50:00,KLO,13.1,10.8,10.1,13.1,65.4,6.8,9.9,973.2,1024.0,1023.7,NaN,NaN,9.3,6.2,6.3,231.0,NaN,22.7,9.1,33.5,32.8,0.0,0.0,1,0.0,300,NaN,NaN,0,LSZH 010650Z 23010KT CAVOK 13/07 Q1024 NOSIG,METAR,NaN,AUTO,LSZH,7.0,230 degrees,10 knots,NaN,NaN,NaN,10000 meters,NaN,NaN,NaN,13.0 C,7.0 C,1024.0 mb,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSZH
3,2023-01-01 07:20:00,2023-01-01 07:50:00,1,18,19,2023-01-01 07:20:00,KLO,12.8,10.7,10.0,12.8,66.3,6.7,9.8,973.4,1024.2,1024.0,NaN,NaN,9.0,5.3,5.4,235.0,NaN,19.4,8.8,32.4,31.7,0.0,0.0,10,8.0,299,NaN,NaN,0,LSZH 010720Z 24007KT CAVOK 13/06 Q1024 NOSIG,METAR,NaN,AUTO,LSZH,7.0,240 degrees,7 knots,NaN,NaN,NaN,10000 meters,NaN,NaN,NaN,13.0 C,6.0 C,1024.0 mb,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LSZH
4,2023-01-01 07:50:00,2023-01-01 08:20:00,5,8,13,2023-01-01 07:50:00,KLO,13.4,12.3,11.7,13.4,64.0,6.7,9.8,973.7,1024.5,1024.2,NaN,NaN,12.7,7.2,7.3,228.0,NaN,26.3,12.2,45.7,43.9,0.0,0.0,40,25.0,300,NaN,